In [1]:
FTRAIN = 'data/training.csv' #getting training data

Загрузка данных: изображений с лицами с меткой 1

In [2]:
import csv

def parse_csv(mark):
    csvfile = open(FTRAIN, 'r')
    reader = csv.reader(csvfile, delimiter='\n')
    my_list = list(reader)
    del my_list[:1]
    array = []
    for entry in my_list:
        entry = ''.join(''.join(entry).partition(' ')[1:])
        entry = entry.split()
        entry.append(mark) #marking images with faces
        array.append(entry)
        
    return array    
    
array = parse_csv("1")

In [3]:
import csv
from random import shuffle

def parse_csv_shuffle(mark):
    csvfile = open(FTRAIN, 'r')
    reader = csv.reader(csvfile, delimiter='\n')
    my_list = list(reader)
    del my_list[:1]
    array = []
    for entry in my_list:
        entry = ''.join(''.join(entry).partition(' ')[1:])
        entry = entry.split()
        shuffle(entry)
        entry.append(mark) #marking images with faces
        array.append(entry)
        
    return array    
    
array_shuffled = parse_csv_shuffle("0")

Подготовка данных

In [6]:
height, width, deph = 96,96,1
batch_size = 128 
num_epochs = 1 
hidden_size = 512 
num_classes = 2 

In [3]:
x_train = []
y_train = []

for entry in array:
    y_train.append(entry[len(entry) - 1])
    x_train.append(entry[0:len(entry) - 1])
    
#for entry in array_shuffled:
#    y_train.append(entry[len(entry) - 1])
#    x_train.append(entry[0:len(entry) - 1])

In [4]:
import numpy

x_train = numpy.array(x_train)
#x_train = x_train.reshape(len(x_train), len(x_train[1]))
x_train = x_train.astype('float32')
x_train /= 225

x_train.shape

(7049, 9215)

In [7]:
from keras.utils import np_utils 

y_train = np_utils.to_categorical(y_train, num_classes) 

Создание модели

In [12]:
from keras.layers import Input, Dense, Dropout, Activation 
from keras.models import Model, Sequential  

def create_model():
    inp = Input(shape=(9215,))
    hidden_1 = Dense(hidden_size, activation ='relu')(inp)
    hidden_2 = Dense(hidden_size, activation = 'relu')(hidden_1) 
    out = Dense(num_classes, activation='softmax')(hidden_2) 
    model = Model(input = inp, output = out)
    return model

def create_model_2():
    model = Sequential()
    model.add(Dense(512,input_shape=(x_train.shape[1],)))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    return model
    
model = create_model_2()

In [13]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_5 (Dense)                  (None, 512)           4718592     dense_input_4[0][0]              
____________________________________________________________________________________________________
activation_4 (Activation)        (None, 512)           0           dense_5[0][0]                    
____________________________________________________________________________________________________
dropout_3 (Dropout)              (None, 512)           0           activation_4[0][0]               
____________________________________________________________________________________________________
dense_6 (Dense)                  (None, 512)           262656      dropout_3[0][0]                  
___________________________________________________________________________________________

In [15]:
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'] )

Обучение модели

In [16]:
model.fit(x_train, y_train, batch_size = batch_size, nb_epoch = num_epochs, verbose=1)

ValueError: Input arrays should have the same number of samples as target arrays. Found 7049 input samples and 14098 target samples.

In [ ]:
model.evaluate(X, y,verbose=1) #оценка модели на тестовом наборе 